#### 1. Import pandas library

In [15]:
import pandas as pd

#### 2. Import users table:

In [16]:
users_table = pd.read_csv("users_table.csv")
users_table.head(2)

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,AccountId,Age,ProfileImageUrl
0,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\r\n\r\n<p>...",0,5007,1920,-1,NaN,NaN
1,2,101,2010-07-19 14:01:36,Geoff Dalgas,2013-11-12 22:07:23,http://stackoverflow.com,"Corvallis, OR",<p>Developer on the StackOverflow team. Find ...,25,3,0,2,37.0,NaN


#### 3. Rename Id column to userId

In [17]:
users_table = users_table.rename(columns={"Id": "userId"})

In [18]:
users_table.head(2)

,userId,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,AccountId,Age,ProfileImageUrl
0,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\r\n\r\n<p>...",0,5007,1920,-1,NaN,NaN
1,2,101,2010-07-19 14:01:36,Geoff Dalgas,2013-11-12 22:07:23,http://stackoverflow.com,"Corvallis, OR",<p>Developer on the StackOverflow team. Find ...,25,3,0,2,37.0,NaN


#### 4. Import posts table:

In [19]:
posts_table = pd.read_csv("posts_table.csv")

In [20]:
posts_table.head(2)

,Id,PostTypeId,AcceptedAnswerId,CreaionDate,Score,ViewCount,Body,OwnerUserId,LasActivityDate,Title,...,AnswerCount,CommentCount,FavoriteCount,LastEditorUserId,LastEditDate,CommunityOwnedDate,ParentId,ClosedDate,OwnerDisplayName,LastEditorDisplayName
0,1,1,15.0,2010-07-19 19:12:12,23,1278.0,<p>How should I elicit prior distributions fro...,8.0,2010-09-15 21:08:26,Eliciting priors from experts,...,5.0,1,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,59.0,2010-07-19 19:12:57,22,8198.0,<p>In many different statistical methods there...,24.0,2012-11-12 09:21:54,What is normality?,...,7.0,1,8.0,88.0,2010-08-07 17:56:44,NaN,NaN,NaN,NaN,NaN


#### 5. Rename Id column to postId and OwnerUserId to userId

In [21]:
posts_table = posts_table.rename(columns={"Id": "postId", }).rename(columns={"OwnerUserID": "userId"})

In [22]:
# I give up, I've used the syntax before and the one following and none are working

posts_table = posts_table.rename(columns=({"Id":"postId"}))
posts_table = posts_table.rename(columns=({"OwnerUserID":"userId"}))

In [23]:
posts_table = posts_table.rename(columns=({"Id":"postId","OwnerUserID":"userId"}))

In [24]:
posts_table.head(2)

,postId,PostTypeId,AcceptedAnswerId,CreaionDate,Score,ViewCount,Body,OwnerUserId,LasActivityDate,Title,...,AnswerCount,CommentCount,FavoriteCount,LastEditorUserId,LastEditDate,CommunityOwnedDate,ParentId,ClosedDate,OwnerDisplayName,LastEditorDisplayName
0,1,1,15.0,2010-07-19 19:12:12,23,1278.0,<p>How should I elicit prior distributions fro...,8.0,2010-09-15 21:08:26,Eliciting priors from experts,...,5.0,1,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,59.0,2010-07-19 19:12:57,22,8198.0,<p>In many different statistical methods there...,24.0,2012-11-12 09:21:54,What is normality?,...,7.0,1,8.0,88.0,2010-08-07 17:56:44,NaN,NaN,NaN,NaN,NaN


#### 6. Define new dataframes for users and posts with the following selected columns:
    **users columns**: userId, Reputation,Views,UpVotes,DownVotes
    **posts columns**: postId, Score,userId,ViewCount,CommentCount

In [25]:
users_df = users_table.loc[:, ["userId", "Reputation", "Views", "UpVotes", "DownVotes"]]
posts_df = posts_table.loc[:, ["userId", "postId", "Score", "ViewCount", "CommentCount"]]


KeyError: "['userId'] not in index"

In [ ]:
users_df.head()

In [ ]:
posts_df.head()

#### 7. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [ ]:
merged_df = pd.merge(users_df, posts_df, on='userId')

#### 8. How many missing values do you have in your merged dataframe? On which columns?

In [ ]:
# No idea, but I would do a .isnull method on all columns to figure it out

#### 9. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before passing to the next step

In [ ]:
# That depends, if there are very few values missing, I'd do the average of the whole column and fill the empty rows with that value,
    # and if the missing values are too many (like over 80% of the rows), I'd drop the column.

#### 10. Adjust the data types in order to avoid future issues. Which ones should be changed? 

In [26]:
print(posts_table.dtypes)
print(users_table.dtypes)

#I'd change numerical variables into categorical

postId                     int64
PostTypeId                 int64
AcceptedAnswerId         float64
CreaionDate               object
Score                      int64
ViewCount                float64
Body                      object
OwnerUserId              float64
LasActivityDate           object
Title                     object
Tags                      object
AnswerCount              float64
CommentCount               int64
FavoriteCount            float64
LastEditorUserId         float64
LastEditDate              object
CommunityOwnedDate        object
ParentId                 float64
ClosedDate                object
OwnerDisplayName          object
LastEditorDisplayName     object
dtype: object
userId               int64
Reputation           int64
CreationDate        object
DisplayName         object
LastAccessDate      object
WebsiteUrl          object
Location            object
AboutMe             object
Views                int64
UpVotes              int64
DownVotes            in